## Summary:
Generate a list of feature layers in an organizations. Generate a list of Web Maps in an organizations. Then print a readout of all the Web Maps that consume a specific feature layer.

### Import the libraries

In [3]:
import sys

from arcgis.gis import GIS
from arcgis.env import active_gis
from arcgis.features import FeatureLayerCollection
from arcgis.mapping import WebMap

### Connect to the organization

In [4]:
g = GIS(profile="your_profile")

### Assign processing environment variable to the current GIS object

In [5]:
active_gis = g

### Customize the PYTHONPATH
Add a path to a local directory containing a number of functions
to assist in various operations.

In [6]:
import sys

sys.path.insert(0, "your_file_pathway/helper_functions/")

In [7]:
import helpers as hpx

### Generate a list of users in the organization

In [8]:
user_list = g.users.search("* AND NOT username:esri*")

for a,b,c in zip(user_list[::3], user_list[1::3], user_list[2::3]):
    print(f"{a.username[:24]:25}{b.username[:24]:25}{c.username[:24]:25}")

your_username           NBAdvanced1              portaladmin              


### Get a user object

In [9]:
target_user = g.users.get("your_username_of_interest")

### Get item inventory

In [10]:
hpx.get_user_items?

In [11]:
target_username, user_item_inv = hpx.get_user_items(target_user, g)

In [12]:
target_username

'your_username'

In [13]:
list(user_item_inv.keys())

['Web Map',
 'GeoJson',
 'OGCFeatureServer',
 'WMS',
 'Map Service',
 'Geoprocessing Service',
 'Symbol Service',
 'Geometry Service',
 'Shapefile',
 'Feature Service',
 'CSV',
 'File Geodatabase',
 'Service Definition',
 'Notebook']

In [ ]:
hpx.print_user_inventory?

In [14]:
hpx.print_user_inventory(user_item_inv)

Web Map
--------------------------------------------------
   Uganda Towns                                      
   Middle Little Missouri SubBasin Wetlands Inventory Map
   Uganda Power Generation Sites 2016                
   Sierra Leone Finance and Agriculture              
   New Mexico County and Land Resource Division Tracts
   New Mexico Land Resources Division Unit Map       


GeoJson
--------------------------------------------------
   Generation_Sites_2016_MEMD                        


OGCFeatureServer
--------------------------------------------------
   SampleWorldCities                                 


WMS
--------------------------------------------------
   SampleWorldCities                                 


Map Service
--------------------------------------------------
   SampleWorldCities                                 
   MLM_Subbasin_and_Wetlands_Inventory               


Geoprocessing Service
--------------------------------------------------
   RasterUtili

### Remove subset of items from the item inventory for demonstration

In [15]:
for k,v in dict(user_item_inv).items():
    v = [z 
         for z in v 
         if z.owner in ["<username_of_interest"]]
    user_item_inv.update({k:v})

### Get list of `Feature Layer Collections` and `Web Maps`

In [16]:
wmaps = [wm 
         for wm in user_item_inv["Web Map"]]
         
flyr_colls = user_item_inv["Feature Service"]

In [17]:
wmaps

[<Item title:"Middle Little Missouri SubBasin Wetlands Inventory Map" type:Web Map owner:your_username>,
 <Item title:"Uganda Power Generation Sites 2016" type:Web Map owner:your_username>,
 <Item title:"Sierra Leone Finance and Agriculture" type:Web Map owner:your_username>,
 <Item title:"New Mexico County and Land Resource Division Tracts" type:Web Map owner:your_username>,
 <Item title:"New Mexico Land Resources Division Unit Map" type:Web Map owner:your_username>]

In [18]:
flyr_colls

[<Item title:"Generation_Sites_2018_MEMD" type:Feature Layer Collection owner:your_username>,
 <Item title:"ImpassibleRoads_Output" type:Feature Layer Collection owner:your_username>,
 <Item title:"ND_NBD_Government_Units" type:Feature Layer Collection owner:your_username>,
 <Item title:"Alternative_Fuel_Stations_ND" type:Feature Layer Collection owner:your_username>,
 <Item title:"Generation_Sites_2016_MEMD" type:Feature Layer Collection owner:your_username>,
 <Item title:"Sierra Leone Finance and Agriculture_WFL1" type:Feature Layer Collection owner:your_username>,
 <Item title:"NM_NPS_LRD_Boundary_Data" type:Feature Layer Collection owner:your_username>,
 <Item title:"Alternative_Fueling_Stations_US" type:Feature Layer Collection owner:your_username>,
 <Item title:"MLM_Subbasin_and_Wetlands_Inventory" type:Feature Layer Collection owner:your_username>]

### Define anonymous function and print whether Items are hosted

In [19]:
is_hosted = lambda x, fli: x in fli.typeKeywords
flyr_dict = {f.title:[f.id, is_hosted("Hosted Service", f)] for f in flyr_colls}

In [20]:
print(f"{'No.':5}{'Feature Layer Collection':42}{'ItemId':35}{'Hosted'}")
print(f"{'-'*3:5}{'-'*40:42}{'-'*32:35}{'-'*6}")
for idx,result in enumerate(flyr_dict.items()):
        print(f"{idx:<5}{result[0][:40]:42}{result[1][0]:35}{result[1][1]}")

No.  Feature Layer Collection                  ItemId                             Hosted
---  ----------------------------------------  --------------------------------   ------
0    Generation_Sites_2018_MEMD                28d0924b80754fa1b640464fd245548e   True
1    ImpassibleRoads_Output                    9884da13003e415489ae183acca5a21e   True
2    ND_NBD_Government_Units                   307bbe3e174546cabad3812cbd502216   True
3    Alternative_Fuel_Stations_ND              a6fb2452b2804f81bf60eae00b8c9416   True
4    Generation_Sites_2016_MEMD                dd90d800f1d948d5a6a79d69217c5dd9   True
5    Sierra Leone Finance and Agriculture_WFL  840e680f1ed64968960a6e2150b4c6f1   True
6    NM_NPS_LRD_Boundary_Data                  22cdfac538744cd28364f62d4892fd0b   True
7    Alternative_Fueling_Stations_US           2f47fdf64cd941d7aec6e803c78dbbe2   True
8    MLM_Subbasin_and_Wetlands_Inventory       67f6ea4409d84334a48ef2c76966b52e   False


### Get a `Feature Layer` and list of `Web Maps` that consume it

In [21]:
hpx.list_webmaps?

In [22]:
flyr_webmaps = hpx.list_webmaps(flyr_colls[6], wmaps)
flyr_webmaps

{'NM_NPS_LRD_Boundary_Data': [<Item title:"New Mexico County and Land Resource Division Tracts" type:Web Map owner:your_username>,
  <Item title:"New Mexico Land Resources Division Unit Map" type:Web Map owner:your_username>]}

In [23]:
for f in flyr_colls:
    flyr_webmaps = hpx.list_webmaps(f, wmaps)
    if not bool(flyr_webmaps[f.title]):
        # print(f"{f.title} Feature Layer Collection consumed in these Web Maps:\n{'-'*80}")
        # print(f"{' '*2}...Zilch!")
        continue
    else:
        print(f"{f.title} Feature Layer Collection consumed in these Web Maps:\n{'-'*95}")
        for flyr,wm in flyr_webmaps.items():
            for wmi in wm:
                print(f"{' '*2}{wmi.title[:55]:60}{wmi.id}")
    print("\n")

Generation_Sites_2016_MEMD Feature Layer Collection consumed in these Web Maps:
-----------------------------------------------------------------------------------------------
  Uganda Power Generation Sites 2016                          347866bdd8b94ee18ac0bd867ea625f9


Sierra Leone Finance and Agriculture_WFL1 Feature Layer Collection consumed in these Web Maps:
-----------------------------------------------------------------------------------------------
  Sierra Leone Finance and Agriculture                        5e4d714aa77349578545fd206f591326


NM_NPS_LRD_Boundary_Data Feature Layer Collection consumed in these Web Maps:
-----------------------------------------------------------------------------------------------
  New Mexico County and Land Resource Division Tracts         a321866dca624f759ef32f6d7ad13973
  New Mexico Land Resources Division Unit Map                 35b3713b643247788651235d67fd2dcb


MLM_Subbasin_and_Wetlands_Inventory Feature Layer Collection consumed in 

### Summary

Notebook demonstrates how to loop through lists of feature layers and web maps to find out what web maps consume what layers.